In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import pprint
import shutil

sys.path.append("pose_estimation")
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

import _init_paths
from core.config import config
from core.config import update_config
from core.config import update_dir
from core.config import get_model_name
from core.loss import JointsMSELoss
from core.function import train
from core.function import validate
from utils.utils import get_optimizer
from utils.utils import save_checkpoint
from utils.utils import create_logger

import dataset
import models

this_dir:  pose_estimation
lib_path:  lib


In [3]:
def parse_args(cfg):
    parser = argparse.ArgumentParser(description='Train keypoints network')
    # general
    parser.add_argument('--cfg',
                        help='experiment configure file name',
                        default=cfg,
                        type=str)

    args, rest = parser.parse_known_args()
    # update config
    update_config(args.cfg)

    # training
    parser.add_argument('--frequent',
                        help='frequency of logging',
                        default=config.PRINT_FREQ,
                        type=int)
    parser.add_argument('--gpus',
                        help='gpus',
                        type=str)
    parser.add_argument('--workers',
                        help='num of dataloader workers',
                        type=int)

    args = parser.parse_args("")

    return args

In [4]:
def reset_config(config, args):
    if args.gpus:
        config.GPUS = args.gpus
    if args.workers:
        config.WORKERS = args.workers

In [5]:
import glob

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [11]:
cfgs = glob.glob(f"experiments/mpii/*/*.yaml")

In [12]:
for cfg in cfgs:
  args = parse_args(cfg)
  reset_config(config, args)
  # Setup model
  model = eval('models.'+config.MODEL.NAME+'.get_pose_net')(
      config, is_train=True
  )
  n_params = count_parameters(model)
  print(f"{cfg} Params: {n_params:,}")

experiments/mpii/resnet101/384x384_d256x3_adam_lr1e-3.yaml Params: 52,991,568
experiments/mpii/resnet101/256x256_d256x3_adam_lr1e-3.yaml Params: 52,991,568
experiments/mpii/resnet152/384x384_d256x3_adam_lr1e-3.yaml Params: 68,635,216
experiments/mpii/resnet152/256x256_d256x3_adam_lr1e-3.yaml Params: 68,635,216
experiments/mpii/resnet50/384x384_d256x3_adam_lr1e-3.yaml Params: 33,999,440
experiments/mpii/resnet50/256x256_d256x3_adam_lr1e-3.yaml Params: 33,999,440
experiments/mpii/cascaded_unet/256x256_d256x3_adam_lr1e-3-unet_x2.yaml Params: 7,087,312
experiments/mpii/cascaded_unet/256x256_d256x3_adam_lr1e-3_large.yaml Params: 10,800,944
experiments/mpii/cascaded_unet/256x256_d256x3_adam_lr1e-3-unet.yaml Params: 3,605,040
experiments/mpii/cascaded_unet/256x256_d256x3_adam_lr1e-3.yaml Params: 3,605,040
experiments/mpii/resnet10/256x256_d256x3_adam_lr1e-3.yaml Params: 5,562,576
experiments/mpii/resnet18/256x256_d256x3_adam_lr1e-3.yaml Params: 11,833,296


In [14]:
import shutil

In [32]:
xs = glob.glob("/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/*/*")

/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/cascaded_pose_resnet_10/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-00-23
/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/cascaded_pose_resnet_10/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-00-22
/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/cascaded_pose_resnet_10/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-00-21
/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/cascaded_pose_resnet_10/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-00-20
/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/cascaded_pose_resnet_10/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-00-19
/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/pose_resnet_18/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-01-00
/home/michael/PoseEstimation/human-pose-estimation.pytorch/log/mpii/pose_resnet_10/256x256_d256x3_adam_lr1e-3-cascaded_2021-08-15-00-26
/ho